# Exercise Set 4

Below are five problems, each worth 1 point. These problems are interleaved with short tutorials on Python. This assignment will be autograded to ensure a quick turnaround. After all of the problems, there are tests associated with your answer which will help you determine if you have solved the problem correctly. If you can run the cell after your answer and not get any errors, then you very likely have gotten the question right. If you have errors, hopefully the error will help you identify the mistake.

Note that just because you don't get errors on the questions that do check your answer doesn't mean that you got the question correct. I have some additional tests held back that I do not show here, though if you pass the ones shown, you will likely pass those as well.

When you are done with the assignment, you should save this notebook manually by clicking on the save button in the toolbar (the floppy disk icon). **Do not rely on autosave. Save manually!** Ensure that you have not renamed the file. **The autograder that is used to grade this notebook requires that the file be named `Exercise_IV.ipynb`.** Once you save the notebook, follow the instructions in the `README.md` file to submit the assignment.

Finally, you are encouraged to add new cells as you go through the notebook and experiment. Any cell that should not be copied or deleted is marked as such. As long as you don't copy or delete the cells marked as such, then you should feel free to experiment as much as you would like with this notebook.

## Introduction to patsy

This assignment will serve as a tutorial on using the package `patsy` to create design matrices. Design matrices are the inputs to most statistical and machine learning models, and using pure Python or `pandas` can make it difficult to get all of the details right. This is particularly true of linear models (like Lasso, Ridge, or OLS), and `patsy` is exceptionally useful for getting the data into the right shape for linear models.

Much of this notebook has been copied, with minor modifications, from the excellent quickstart guide at `patsy`'s documentation. You can find the original documentation [here](https://patsy.readthedocs.io/en/latest/overview.html). The exercises are novel for this assignment.

`patsy` is a Python package for describing statistical models
(especially linear models, or models that have a linear component)
and building design matrices. It is closely inspired by and compatible
with the [formula](http://cran.r-project.org/doc/manuals/R-intro.html#Formulae-for-statistical-models) mini-language used in [R](http://www.r-project.org/) and [S](https://secure.wikimedia.org/wikipedia/en/wiki/S_programming_language).

For instance, if we have some variable `y`, and we want to regress it
against some other variables `x`, `a`, `b`, and the [interaction](https://secure.wikimedia.org/wikipedia/en/wiki/Interaction_%28statistics%29)
of `a` and `b`, then we simply write::

``` python
patsy.dmatrices("y ~ x + a + b + a:b", data)
```

and `patsy` takes care of building appropriate matrices. Furthermore,
it:

* Allows data transformations to be specified using arbitrary Python
  code: instead of ``x``, we could have written ``log(x)``, ``(x >
  0)``, or even ``log(x) if x > 1e-5 else log(1e-5)``,
* Provides a range of convenient options for coding [categorical](https://secure.wikimedia.org/wikipedia/en/wiki/Level_of_measurement#Nominal_scale)
  variables, including automatic detection and removal of
  redundancies,
* Knows how to apply 'the same' transformation used on original data
  to new data, even for tricky transformations like centering or
  standardization (critical if you want to use your model to make
  predictions),
* Has an incremental mode to handle data sets which are too large to
  fit into memory at one time,
* Provides a language for symbolic, human-readable specification of
  linear constraint matrices,
* Has a thorough test suite (>97% statement coverage) and solid
  underlying theory, allowing it to correctly handle corner cases that
  even R gets wrong, and
* Features a simple API for integration into statistical packages.

What Patsy *won't* do is, well, statistics --- it just lets you
describe models in general terms. It doesn't know or care whether you
ultimately want to do linear regression, time-series analysis, or fit
a forest of [decision trees](https://secure.wikimedia.org/wikipedia/en/wiki/Decision_tree_learning),
and it certainly won't do any of those things for you --- it just
gives a high-level language for describing which factors you want your
underlying model to take into account.

First, let’s import stuff and get some data to work with:

In [200]:
import numpy as np
import pandas as pd
from patsy import dmatrices, dmatrix, demo_data, build_design_matrices
from sklearn.model_selection import train_test_split

The function `demo_data` allows us to get some data to play around with.

In [201]:
data = pd.DataFrame(demo_data("a", "b", "x1", "x2", "y", "z_column"))

## Building design matrices

Now we have some `data` stored in a dataframe. Let's take a look.

In [202]:
data

,a,b,x1,x2,y,z_column
0,a1,b1,1.764052,-0.103219,1.494079,2.269755
1,a1,b2,0.400157,0.410599,-0.205158,-1.454366
2,a2,b1,0.978738,0.144044,0.313068,0.045759
3,a2,b2,2.240893,1.454274,-0.854096,-0.187184
4,a1,b1,1.867558,0.761038,-2.552990,1.532779
5,a1,b2,-0.977278,0.121675,0.653619,1.469359
6,a2,b1,0.950088,0.443863,0.864436,0.154947
7,a2,b2,-0.151357,0.333674,-0.742165,0.378163


As you can see, `demo_data()` gave us a mix of categorical and numerical variables.

Now, let’s generate design matrices suitable for regressing `y` onto `x1` and `x2`.

In [203]:
dmatrices("y ~ x1 + x2", data, return_type="dataframe")

(          y
 0  1.494079
 1 -0.205158
 2  0.313068
 3 -0.854096
 4 -2.552990
 5  0.653619
 6  0.864436
 7 -0.742165,
    Intercept        x1        x2
 0        1.0  1.764052 -0.103219
 1        1.0  0.400157  0.410599
 2        1.0  0.978738  0.144044
 3        1.0  2.240893  1.454274
 4        1.0  1.867558  0.761038
 5        1.0 -0.977278  0.121675
 6        1.0  0.950088  0.443863
 7        1.0 -0.151357  0.333674)

Notice that there are three inputs to the `dmatrices` function. The first is the formula that we are going to use. The second is our data source, and the third is telling `dmatrices` that we want a pandas DataFrame back. Actually, technically we want two dataframes back, one for the `y` (or the dependent variable) and one for the `X` (the set of independent variables). We can save these two things as follows:

In [204]:
y, X = dmatrices("y ~ x1 + x2", data, return_type="dataframe")

The return value is a Python tuple containing two DesignMatrix objects, the first representing the left-hand side of our formula, and the second representing the right-hand side. Notice that an intercept term was automatically added to the right-hand side. These are just ordinary pandas dataframces with some extra metadata so that we can re-use the transformation process.

In [205]:
y

,y
0,1.494079
1,-0.205158
2,0.313068
3,-0.854096
4,-2.552990
5,0.653619
6,0.864436
7,-0.742165


In [206]:
X

,Intercept,x1,x2
0,1.0,1.764052,-0.103219
1,1.0,0.400157,0.410599
2,1.0,0.978738,0.144044
3,1.0,2.240893,1.454274
4,1.0,1.867558,0.761038
5,1.0,-0.977278,0.121675
6,1.0,0.950088,0.443863
7,1.0,-0.151357,0.333674


Note, we can use whatever data we want in for either the left or right hand side of things.

In [207]:
y, X = dmatrices("z_column ~ y + a", data, return_type="dataframe")
display(y)
display(X)

,z_column
0,2.269755
1,-1.454366
2,0.045759
3,-0.187184
4,1.532779
5,1.469359
6,0.154947
7,0.378163


,Intercept,a[T.a2],y
0,1.0,0.0,1.494079
1,1.0,0.0,-0.205158
2,1.0,1.0,0.313068
3,1.0,1.0,-0.854096
4,1.0,0.0,-2.552990
5,1.0,0.0,0.653619
6,1.0,1.0,0.864436
7,1.0,1.0,-0.742165


As you can see, the `y` design matrix contains the variable `z_column` and the `X` design matrix contains `Intercept`, `y`, and a column called `a[T.a2]`. This column `a[T.a2]` is due to `a` being a categorical variable, and we will explore categorical variables in a little while.

### Problem #1 - 1 point

Use the following data set, `data_p1`, to create design matrices, `y_p1` and `X_p1`.

In [208]:
# This is a read only cell.
data_p1 = pd.DataFrame(demo_data("c1", "c2", "c3", "r1", "r2", "r3", "y1"))

Let's see what is in `data_p1`.

In [209]:
data_p1

,c1,c2,c3,r1,r2,r3,y1
0,c11,c21,c31,1.764052,-0.103219,1.494079,2.269755
1,c11,c21,c32,0.400157,0.410599,-0.205158,-1.454366
2,c11,c22,c31,0.978738,0.144044,0.313068,0.045759
3,c11,c22,c32,2.240893,1.454274,-0.854096,-0.187184
4,c12,c21,c31,1.867558,0.761038,-2.552990,1.532779
5,c12,c21,c32,-0.977278,0.121675,0.653619,1.469359
6,c12,c22,c31,0.950088,0.443863,0.864436,0.154947
7,c12,c22,c32,-0.151357,0.333674,-0.742165,0.378163


Create design matrices `y_p1` and `X_p1` where `y_p1` is the variable `y1` from data set `data_p1`, and `X_p1` includes only the columns `r1`, `r3`, and an intercept column.

In [210]:
import pandas as pd
from patsy import dmatrix

# Sample data provided
data_p1 = pd.DataFrame({
    "c1": ["c11", "c11", "c11", "c11", "c12", "c12", "c12", "c12"],
    "c2": ["c21", "c21", "c22", "c22", "c21", "c21", "c22", "c22"],
    "c3": ["c31", "c32", "c31", "c32", "c31", "c32", "c31", "c32"],
    "r1": [1.764052, 0.400157, 0.978738, 2.240893, 1.867558, -0.977278, 0.950088, -0.151357],
    "r2": [-0.103219, 0.410599, 0.144044, 1.454274, 0.761038, 0.121675, 0.443863, 0.333674],
    "r3": [1.494079, -0.205158, 0.313068, -0.854096, -2.552990, 0.653619, 0.864436, -0.742165],
    "y1": [2.269755, -1.454366, 0.045759, -0.187184, 1.532779, 1.469359, 0.154947, 0.378163]
})

# Create y_p1
y_p1 = data_p1[["y1"]]

# Create X_p1
X_p1 = dmatrix("r1 + r3 + 1", data=data_p1, return_type="dataframe")

# The output matrices are now y_p1 and X_p1


In [211]:
# THIS IS A GRADING CELL. DO NOT EDIT AND DO NOT COPY.
from nose.tools import assert_equal, assert_true
import numpy as np
print("You have set y_p1 as:")
display(y_p1)
print("You have set X_p1 as:")
display(X_p1)
assert_true(isinstance(y_p1, pd.DataFrame))
assert_true("Intercept" in X_p1 and "r1" in X_p1 and "r3" in X_p1 and len(X_p1.columns) == 3)
assert_true(y_p1.loc[[0]].round(1).isin([2.3]).any().any())
assert_true(y_p1.loc[[3]].round(1).isin([-0.2]).any().any())
assert_true(X_p1.loc[[0], :].round(1).isin([1.5]).any().any())
assert_true(X_p1.loc[[3], :].round(1).isin([2.2]).any().any())

You have set y_p1 as:


,y1
0,2.269755
1,-1.454366
2,0.045759
3,-0.187184
4,1.532779
5,1.469359
6,0.154947
7,0.378163


You have set X_p1 as:


,Intercept,r1,r3
0,1.0,1.764052,1.494079
1,1.0,0.400157,-0.205158
2,1.0,0.978738,0.313068
3,1.0,2.240893,-0.854096
4,1.0,1.867558,-2.552990
5,1.0,-0.977278,0.653619
6,1.0,0.950088,0.864436
7,1.0,-0.151357,-0.742165


## Transforming the Data with Patsy

If you just want the design matrix alone, without the `y` values, use `dmatrix()` and leave off the `y ~` part at the beginning:

In [212]:
dmatrix("x1 + x2", data, return_type="dataframe")

,Intercept,x1,x2
0,1.0,1.764052,-0.103219
1,1.0,0.400157,0.410599
2,1.0,0.978738,0.144044
3,1.0,2.240893,1.454274
4,1.0,1.867558,0.761038
5,1.0,-0.977278,0.121675
6,1.0,0.950088,0.443863
7,1.0,-0.151357,0.333674


We’ll use `dmatrix` for the rest of the examples, since seeing the outcome matrix over and over would get boring. This matrix’s metadata is stored in an extra attribute called `.design_info`, which is a DesignInfo object that lets you re-use the data transformation that you made.

In [213]:
d = dmatrix("x1 + x2", data, return_type="dataframe")

In [214]:
d.design_info

DesignInfo(['Intercept', 'x1', 'x2'],
           factor_infos={EvalFactor('x1'): FactorInfo(factor=EvalFactor('x1'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('x2'): FactorInfo(factor=EvalFactor('x2'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('x1')]),
                                      [SubtermInfo(factors=(EvalFactor('x1'),),
                                                   contrast_matrices={},
     

Usually the intercept is useful, but if we don’t want it we can get rid of it by adding a `+ 0` to the right hand side.

In [215]:
dmatrix("x1 + x2 + 0", data, return_type="dataframe")

,x1,x2
0,1.764052,-0.103219
1,0.400157,0.410599
2,0.978738,0.144044
3,2.240893,1.454274
4,1.867558,0.761038
5,-0.977278,0.121675
6,0.950088,0.443863
7,-0.151357,0.333674


We can also make the intercept explicit by adding a `+ 1` to the right hand side. Note that `dmatrix("x1 + x2", data, return_type="dataframe")` and `dmatrix("x1 + x2 + 1", data, return_type="dataframe")` return the same dataframes, but one makes the intercept a little more obvious.

In [216]:
dmatrix("x1 + x2 + 1", data, return_type="dataframe")

,Intercept,x1,x2
0,1.0,1.764052,-0.103219
1,1.0,0.400157,0.410599
2,1.0,0.978738,0.144044
3,1.0,2.240893,1.454274
4,1.0,1.867558,0.761038
5,1.0,-0.977278,0.121675
6,1.0,0.950088,0.443863
7,1.0,-0.151357,0.333674


We can transform variables using arbitrary Python code:

In [217]:
dmatrix("x1 + np.log(x2 + 10)", data, return_type="dataframe")

,Intercept,x1,np.log(x2 + 10)
0,1.0,1.764052,2.292210
1,1.0,0.400157,2.342824
2,1.0,0.978738,2.316887
3,1.0,2.240893,2.438363
4,1.0,1.867558,2.375932
5,1.0,-0.977278,2.314679
6,1.0,0.950088,2.346015
7,1.0,-0.151357,2.335408


This allows us to easily do transformations to the data. Suppose that we need to use a log transform for both the dependent variable `y` and the the independent variable `x2`, but we want to leave `x1` as is. Since `y` and `x2` generally can be negative, we will first add `10` to it to make sure that these variables are never negative. This can all be done together like so:

In [218]:
y, X = dmatrices("np.log(y + 10) ~ x1 + np.log(x2 + 10)", data, return_type="dataframe")
display(y)
display(X)

,np.log(y + 10)
0,2.441832
1,2.281856
2,2.333412
3,2.213306
4,2.007813
5,2.365900
6,2.385495
7,2.225470


,Intercept,x1,np.log(x2 + 10)
0,1.0,1.764052,2.292210
1,1.0,0.400157,2.342824
2,1.0,0.978738,2.316887
3,1.0,2.240893,2.438363
4,1.0,1.867558,2.375932
5,1.0,-0.977278,2.314679
6,1.0,0.950088,2.346015
7,1.0,-0.151357,2.335408


Notice that `np.log` is being pulled out of the environment where `dmatrix()` was called – `np.log` is accessible because we did `import numpy as np` up above. Any functions that you have loaded up can also be used inside the formula passed to `dmatrix()`. For example, if you wanted to square a column using the function `np.square` you could:

In [219]:
dmatrix("np.square(x2)", data, return_type="dataframe")

,Intercept,np.square(x2)
0,1.0,0.010654
1,1.0,0.168591
2,1.0,0.020749
3,1.0,2.114911
4,1.0,0.579178
5,1.0,0.014805
6,1.0,0.197015
7,1.0,0.111339


### Problem #2 - 1 point

Use the following data set, `data_p2`, to create design matrices, `y_p2` and `X_p2`.

In [220]:
# This is a read only cell.
data_p2 = pd.DataFrame(demo_data("c1", "c2", "c3", "r1", "r2", "r3", "y1"))

Let's see what is in `data_p2`.

In [221]:
data_p2

,c1,c2,c3,r1,r2,r3,y1
0,c11,c21,c31,1.764052,-0.103219,1.494079,2.269755
1,c11,c21,c32,0.400157,0.410599,-0.205158,-1.454366
2,c11,c22,c31,0.978738,0.144044,0.313068,0.045759
3,c11,c22,c32,2.240893,1.454274,-0.854096,-0.187184
4,c12,c21,c31,1.867558,0.761038,-2.552990,1.532779
5,c12,c21,c32,-0.977278,0.121675,0.653619,1.469359
6,c12,c22,c31,0.950088,0.443863,0.864436,0.154947
7,c12,c22,c32,-0.151357,0.333674,-0.742165,0.378163


Create design matrices `y_p2` and `X_p2` where `y_p2` is the square of the variable `y1` from data set `data_p2`, and `X_p2` only includes the log of `r1 + 10` and `r1` raised to the third power (i.e., `r1^3` in more traditional notation). Note that there should be exactly two columns and both are derived from the variable `r1`. Also, there should not be an intercept column. You may want to look up the numpy function `np.power` to raise `r1` to the third power. The documentation for `np.power` can be found [here](https://numpy.org/doc/stable/reference/generated/numpy.power.html).

In [222]:
import pandas as pd
import numpy as np

# Given dataset (assuming demo_data function is already defined and available)
data_p2 = pd.DataFrame({
    'c1': ['c11', 'c11', 'c11', 'c11', 'c12', 'c12', 'c12', 'c12'],
    'c2': ['c21', 'c21', 'c22', 'c22', 'c21', 'c21', 'c22', 'c22'],
    'c3': ['c31', 'c32', 'c31', 'c32', 'c31', 'c32', 'c31', 'c32'],
    'r1': [1.764052, 0.400157, 0.978738, 2.240893, 1.867558, -0.977278, 0.950088, -0.151357],
    'r2': [-0.103219, 0.410599, 0.144044, 1.454274, 0.761038, 0.121675, 0.443863, 0.333674],
    'r3': [1.494079, -0.205158, 0.313068, -0.854096, -2.552990, 0.653619, 0.864436, -0.742165],
    'y1': [2.269755, -1.454366, 0.045759, -0.187184, 1.532779, 1.469359, 0.154947, 0.378163]
})

# Creating y_p2 matrix
y_p2 = data_p2['y1']**2

# Convert y_p2 from Series to DataFrame
y_p2 = y_p2.to_frame(name='y1_squared')

# Creating X_p2 matrix
X_p2 = pd.DataFrame({
    'log_r1+10': np.log(data_p2['r1'] + 10),
    'r1^3': np.power(data_p2['r1'], 3)
})

# Displaying the matrices for verification
print("y_p2:")
print(y_p2)
print("\nX_p2:")
print(X_p2)


y_p2:
   y1_squared
0    5.151788
1    2.115180
2    0.002094
3    0.035038
4    2.349411
5    2.159016
6    0.024009
7    0.143007

X_p2:
   log_r1+10       r1^3
0   2.465048   5.489517
1   2.341821   0.064075
2   2.395960   0.937561
3   2.504782  11.252872
4   2.473808   6.513618
5   2.199746  -0.933371
6   2.393347   0.857613
7   2.287334  -0.003467


In [223]:
# THIS IS A GRADING CELL. DO NOT EDIT AND DO NOT COPY.
from nose.tools import assert_equal, assert_true
import numpy as np
print("You have set y_p2 as:")
display(y_p2)
print("You have set X_p2 as:")
display(X_p2)
assert_true(isinstance(y_p2, pd.DataFrame))
assert_true("Intercept" not in X_p2 and len(X_p2.columns) == 2)
assert_true(y_p2.loc[[0]].round(1).isin([5.2]).any().any())
assert_true(y_p2.loc[[3]].round(1).isin([0.0]).any().any())
assert_true(X_p2.loc[[0], :].round(1).isin([2.5]).any().any())
assert_true(X_p2.loc[[3], :].round(1).isin([11.3]).any().any())

You have set y_p2 as:


,y1_squared
0,5.151788
1,2.115180
2,0.002094
3,0.035038
4,2.349411
5,2.159016
6,0.024009
7,0.143007


You have set X_p2 as:


,log_r1+10,r1^3
0,2.465048,5.489517
1,2.341821,0.064075
2,2.395960,0.937561
3,2.504782,11.252872
4,2.473808,6.513618
5,2.199746,-0.933371
6,2.393347,0.857613
7,2.287334,-0.003467


## Stateful Transforms with Patsy

Patsy has some useful transformation functions “built in”, that are automatically accessible to your code. For example, you can `standardize()`, make your data mean `0` and standard deviation `1`, or `center()`, make your data mean `0` but leave the standard deviation alone, with a simple function:

In [224]:
X = dmatrix("center(2*x1) + standardize(2*x2)", data, return_type="dataframe")
display(X)

,Intercept,center(2 * x1),standardize(2 * x2)
0,1.0,1.759892,-1.217011
1,1.0,-0.967899,-0.077914
2,1.0,0.189263,-0.668847
3,1.0,2.713573,2.235840
4,1.0,1.966903,0.698985
5,1.0,-3.722769,-0.718437
6,1.0,0.131964,-0.004168
7,1.0,-2.070927,-0.248449


In [225]:
X.mean()

Intercept              1.000000e+00
center(2 * x1)         1.665335e-16
standardize(2 * x2)    1.040834e-17
dtype: float64

In [226]:
X.std()

Intercept              0.000000
center(2 * x1)         2.186557
standardize(2 * x2)    1.069045
dtype: float64

This flexibility does create problems in one case, though – because whatever you write in-between the `+` signs is interpreted as Python code, you do in fact have to write valid Python code. And this can be tricky if your variable names have funny characters in them, like whitespace or punctuation. Fortunately, `patsy` has a builtin “transformation” called `Q()` that lets you “quote” such variables:

In [227]:
weird_data = pd.DataFrame(demo_data("weird column!", "x1"))

In [228]:
weird_data

,weird column!,x1
0,1.764052,-0.977278
1,0.400157,0.950088
2,0.978738,-0.151357
3,2.240893,-0.103219
4,1.867558,0.410599


In [229]:
# This will give an error
dmatrix("weird column! + x1", weird_data)

PatsyError: error tokenizing input (maybe an unclosed string?)
    weird column! + x1
                ^

In [230]:
# ...but this works:
dmatrix("Q('weird column!') + x1", weird_data, return_type="dataframe")

,Intercept,Q('weird column!'),x1
0,1.0,1.764052,-0.977278
1,1.0,0.400157,0.950088
2,1.0,0.978738,-0.151357
3,1.0,2.240893,-0.103219
4,1.0,1.867558,0.410599


Arithmetic transformations are also possible, but you’ll need to “protect” them by wrapping them in `I()`, so that Patsy knows that you really do want `+` to mean addition:

In [231]:
dmatrix("I(x1 + x2)", data, return_type="dataframe")

,Intercept,I(x1 + x2)
0,1.0,1.660833
1,1.0,0.810756
2,1.0,1.122782
3,1.0,3.695167
4,1.0,2.628596
5,1.0,-0.855603
6,1.0,1.393952
7,1.0,0.182317


Compared to `x1 + x2`:

In [232]:
dmatrix("x1 + x2", data, return_type="dataframe")

,Intercept,x1,x2
0,1.0,1.764052,-0.103219
1,1.0,0.400157,0.410599
2,1.0,0.978738,0.144044
3,1.0,2.240893,1.454274
4,1.0,1.867558,0.761038
5,1.0,-0.977278,0.121675
6,1.0,0.950088,0.443863
7,1.0,-0.151357,0.333674


### Problem #3 - 1 point

Use the following data set, `data_p3`, to create the design matrix `X_p3`.

In [233]:
# This is a read only cell.
data_p3 = pd.DataFrame(demo_data("c1", "c2", "c3", "r1", "r2", "r3", "y1"))
data_p3.rename(columns={"r3": "0levels"}, inplace=True)

Let's see what is in `data_p3`.

In [234]:
data_p3

,c1,c2,c3,r1,r2,0levels,y1
0,c11,c21,c31,1.764052,-0.103219,1.494079,2.269755
1,c11,c21,c32,0.400157,0.410599,-0.205158,-1.454366
2,c11,c22,c31,0.978738,0.144044,0.313068,0.045759
3,c11,c22,c32,2.240893,1.454274,-0.854096,-0.187184
4,c12,c21,c31,1.867558,0.761038,-2.552990,1.532779
5,c12,c21,c32,-0.977278,0.121675,0.653619,1.469359
6,c12,c22,c31,0.950088,0.443863,0.864436,0.154947
7,c12,c22,c32,-0.151357,0.333674,-0.742165,0.378163


Unlike other problems, `data_p3` contains the column `0levels`. Note that like the `weird column!` example above, column names that begin with a number are invalid and must use the `Q()` function.

Create the design matrix `X_p3` (note that you do not have to create a y design matrix in this problem) where `X_p3` only includes a single column which is the sum of the columns `0levels` and `r1`. Note, there should not be an intercept column.

In [235]:
from patsy import dmatrix
import pandas as pd

# Given data
data_p3 = pd.DataFrame({
    "c1": ["c11", "c11", "c11", "c11", "c12", "c12", "c12", "c12"],
    "c2": ["c21", "c21", "c22", "c22", "c21", "c21", "c22", "c22"],
    "c3": ["c31", "c32", "c31", "c32", "c31", "c32", "c31", "c32"],
    "r1": [1.764052, 0.400157, 0.978738, 2.240893, 1.867558, -0.977278, 0.950088, -0.151357],
    "r2": [-0.103219, 0.410599, 0.144044, 1.454274, 0.761038, 0.121675, 0.443863, 0.333674],
    "0levels": [1.494079, -0.205158, 0.313068, -0.854096, -2.552990, 0.653619, 0.864436, -0.742165],
    "y1": [2.269755, -1.454366, 0.045759, -0.187184, 1.532779, 1.469359, 0.154947, 0.378163]
})

# Creating the design matrix using dmatrix from patsy
X_p3 = dmatrix("0 + Q('0levels') + r1", data=data_p3, return_type="dataframe")

# Calculating the sum of the two columns '0levels' and 'r1' and storing it in a new column named 'sum'
X_p3["sum"] = X_p3["Q('0levels')"] + X_p3["r1"]

# Retaining only the 'sum' column as required
X_p3 = X_p3[["sum"]]

# Displaying the resulting matrix X_p3
X_p3


,sum
0,3.258131
1,0.194999
2,1.291806
3,1.386797
4,-0.685432
5,-0.323659
6,1.814524
7,-0.893522


In [236]:
# THIS IS A GRADING CELL. DO NOT EDIT AND DO NOT COPY.
from nose.tools import assert_equal, assert_true
import numpy as np
print("You have set X_p3 as:")
display(X_p3)
assert_true("Intercept" not in X_p3 and len(X_p3.columns) == 1)
assert_true(X_p3.loc[[0], :].round(1).isin([3.3]).any().any())
assert_true(X_p3.loc[[3], :].round(1).isin([1.4]).any().any())

You have set X_p3 as:


,sum
0,3.258131
1,0.194999
2,1.291806
3,1.386797
4,-0.685432
5,-0.323659
6,1.814524
7,-0.893522


## Coding Categorical Data with Patsy

Patsy becomes particularly useful when you have categorical data. If you use a predictor that has a categorical type (e.g. strings or bools), it will be automatically coded. Patsy automatically chooses an appropriate way to code categorical data to avoid producing a redundant, overdetermined model.

If there is just one categorical variable alone, the default is to dummy code it if you don't have an intercept:

In [237]:
dmatrix("0 + a", data, return_type="dataframe")

,a[a1],a[a2]
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,1.0,0.0
5,1.0,0.0
6,0.0,1.0
7,0.0,1.0


But if you did that and put the intercept back in, you’d get a redundant model. So if the intercept is present, `patsy` automatically drops one of the columns so that your model is not overspecified:

In [238]:
dmatrix("a", data, return_type="dataframe")

,Intercept,a[T.a2]
0,1.0,0.0
1,1.0,0.0
2,1.0,1.0
3,1.0,1.0
4,1.0,0.0
5,1.0,0.0
6,1.0,1.0
7,1.0,1.0


The `T.` notation is there to remind you that these columns are treatment coded (i.e., it dropped one of the columns).

Interactions are also easy – they represent the \product of all the factors involved. Here’s a dummy coding of each combination of values taken by a and b:

In [239]:
dmatrix("0 + a:b", data, return_type="dataframe")

,a[a1]:b[b1],a[a2]:b[b1],a[a1]:b[b2],a[a2]:b[b2]
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,1.0


But interactions also know how to use drop necessary columns (i.e., use *contrast coding*) to avoid redundancy. If you have both main effects and interactions in a model, then Patsy goes from lower-order effects to higher-order effects, adding in just enough columns to produce a well-defined model. The result is that each set of columns measures the additional contribution of this effect – just what you want for a standard regression model:

In [240]:
dmatrix("a + b + a:b", data, return_type="dataframe")

,Intercept,a[T.a2],b[T.b2],a[T.a2]:b[T.b2]
0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0
2,1.0,1.0,0.0,0.0
3,1.0,1.0,1.0,1.0
4,1.0,0.0,0.0,0.0
5,1.0,0.0,1.0,0.0
6,1.0,1.0,0.0,0.0
7,1.0,1.0,1.0,1.0


Since this is so common, there’s a convenient short-hand:

In [241]:
dmatrix("a*b", data, return_type="dataframe")

,Intercept,a[T.a2],b[T.b2],a[T.a2]:b[T.b2]
0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0
2,1.0,1.0,0.0,0.0
3,1.0,1.0,1.0,1.0
4,1.0,0.0,0.0,0.0
5,1.0,0.0,1.0,0.0
6,1.0,1.0,0.0,0.0
7,1.0,1.0,1.0,1.0


You can even write interactions between categorical and numerical variables. Here we fit two different slope coefficients for x1; one for the a1 group, and one for the a2 group:

In [242]:
dmatrix("a:x1", data, return_type="dataframe")

,Intercept,a[a1]:x1,a[a2]:x1
0,1.0,1.764052,0.000000
1,1.0,0.400157,0.000000
2,1.0,0.000000,0.978738
3,1.0,0.000000,2.240893
4,1.0,1.867558,0.000000
5,1.0,-0.977278,-0.000000
6,1.0,0.000000,0.950088
7,1.0,-0.000000,-0.151357


The same redundancy avoidance code works here, so if you’d rather have treatment-coded slopes (one slope for the `a1` group, and a second for the difference between the `a1` and `a2` group slopes), then you can request it like this:

In [243]:
dmatrix("x1 + a:x1", data, return_type="dataframe")

,Intercept,x1,a[T.a2]:x1
0,1.0,1.764052,0.000000
1,1.0,0.400157,0.000000
2,1.0,0.978738,0.978738
3,1.0,2.240893,2.240893
4,1.0,1.867558,0.000000
5,1.0,-0.977278,-0.000000
6,1.0,0.950088,0.950088
7,1.0,-0.151357,-0.151357


However, often, we have categorical data where the categories are represented by numbers. This presents a problem for `patsy` because if a column is entirely numeric, it has no way to know that it should treat it as a category. Take the following modification to the `data` dataframe, for example.

In [244]:
np.random.seed(204)
data["numeric_categorical"] = np.random.choice([1,2,3,4], 8)
display(data)

,a,b,x1,x2,y,z_column,numeric_categorical
0,a1,b1,1.764052,-0.103219,1.494079,2.269755,2
1,a1,b2,0.400157,0.410599,-0.205158,-1.454366,1
2,a2,b1,0.978738,0.144044,0.313068,0.045759,2
3,a2,b2,2.240893,1.454274,-0.854096,-0.187184,2
4,a1,b1,1.867558,0.761038,-2.552990,1.532779,2
5,a1,b2,-0.977278,0.121675,0.653619,1.469359,3
6,a2,b1,0.950088,0.443863,0.864436,0.154947,1
7,a2,b2,-0.151357,0.333674,-0.742165,0.378163,4


`dmatrix` will assume that `numeric_categorical` is not a categorical value:

In [245]:
dmatrix("numeric_categorical", data, return_type="dataframe")

,Intercept,numeric_categorical
0,1.0,2.0
1,1.0,1.0
2,1.0,2.0
3,1.0,2.0
4,1.0,2.0
5,1.0,3.0
6,1.0,1.0
7,1.0,4.0


However, you can force it to consider it categorical using the `C()` function.

In [246]:
dmatrix("C(numeric_categorical)", data, return_type="dataframe")

,Intercept,C(numeric_categorical)[T.2],C(numeric_categorical)[T.3],C(numeric_categorical)[T.4]
0,1.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0
3,1.0,1.0,0.0,0.0
4,1.0,1.0,0.0,0.0
5,1.0,0.0,1.0,0.0
6,1.0,0.0,0.0,0.0
7,1.0,0.0,0.0,1.0


You can use the `C()` function to do anything you would normally do with a categorical variable, such as interactions.

In [247]:
dmatrix("0 + x1 + x2*C(numeric_categorical)", data, return_type="dataframe")

,C(numeric_categorical)[1],C(numeric_categorical)[2],C(numeric_categorical)[3],C(numeric_categorical)[4],x1,x2,x2:C(numeric_categorical)[T.2],x2:C(numeric_categorical)[T.3],x2:C(numeric_categorical)[T.4]
0,0.0,1.0,0.0,0.0,1.764052,-0.103219,-0.103219,-0.000000,-0.000000
1,1.0,0.0,0.0,0.0,0.400157,0.410599,0.000000,0.000000,0.000000
2,0.0,1.0,0.0,0.0,0.978738,0.144044,0.144044,0.000000,0.000000
3,0.0,1.0,0.0,0.0,2.240893,1.454274,1.454274,0.000000,0.000000
4,0.0,1.0,0.0,0.0,1.867558,0.761038,0.761038,0.000000,0.000000
5,0.0,0.0,1.0,0.0,-0.977278,0.121675,0.000000,0.121675,0.000000
6,1.0,0.0,0.0,0.0,0.950088,0.443863,0.000000,0.000000,0.000000
7,0.0,0.0,0.0,1.0,-0.151357,0.333674,0.000000,0.000000,0.333674


### Problem #4 - 1 point

Use the following data set, `data_p4`, to create the design matrix `X_p4`.

In [248]:
# This is a read only cell.
data_p4 = pd.DataFrame(demo_data("c1", "c2", "c3", "r1", "r2", "y1"))
np.random.seed(204)
data_p4["c4"] = np.random.choice([1,2,3,4], 8)

Let's see what is in `data_p4`.

In [249]:
data_p4

,c1,c2,c3,r1,r2,y1,c4
0,c11,c21,c31,1.764052,-0.103219,1.494079,2
1,c11,c21,c32,0.400157,0.410599,-0.205158,1
2,c11,c22,c31,0.978738,0.144044,0.313068,2
3,c11,c22,c32,2.240893,1.454274,-0.854096,2
4,c12,c21,c31,1.867558,0.761038,-2.552990,2
5,c12,c21,c32,-0.977278,0.121675,0.653619,3
6,c12,c22,c31,0.950088,0.443863,0.864436,1
7,c12,c22,c32,-0.151357,0.333674,-0.742165,4


Unlike other problems, `data_p4` contains the column `c4` which is a categorical variable but the categories are numeric. Therefore, `patsy` will not automatically recognize it is categorical, and you must use the `C()` function.

Create the design matrix `X_p4` (note that you do not have to create a y design matrix in this problem) where `X_p4` includes an Intercept column, dummy coding for the variable `c4` (with a redundant column dropped, so there should only be three columns for `c4` even though there are four categories), the variable `r1`, and an interaction between `c4` and `r1`. All in all, there should be eight columns, and you should include the intercept.

In [250]:
from patsy import dmatrix

# Create the design matrix X_p4 with the Intercept
X_p4 = dmatrix("~ 1 + C(c4) + r1 + C(c4)*r1", data_p4, return_type="dataframe")

# Display the resulting X_p4
display(X_p4)


,Intercept,C(c4)[T.2],C(c4)[T.3],C(c4)[T.4],r1,C(c4)[T.2]:r1,C(c4)[T.3]:r1,C(c4)[T.4]:r1
0,1.0,1.0,0.0,0.0,1.764052,1.764052,0.000000,0.000000
1,1.0,0.0,0.0,0.0,0.400157,0.000000,0.000000,0.000000
2,1.0,1.0,0.0,0.0,0.978738,0.978738,0.000000,0.000000
3,1.0,1.0,0.0,0.0,2.240893,2.240893,0.000000,0.000000
4,1.0,1.0,0.0,0.0,1.867558,1.867558,0.000000,0.000000
5,1.0,0.0,1.0,0.0,-0.977278,-0.000000,-0.977278,-0.000000
6,1.0,0.0,0.0,0.0,0.950088,0.000000,0.000000,0.000000
7,1.0,0.0,0.0,1.0,-0.151357,-0.000000,-0.000000,-0.151357


In [251]:
# THIS IS A GRADING CELL. DO NOT EDIT AND DO NOT COPY.
from nose.tools import assert_equal, assert_true
import numpy as np
print("You have set X_p4 as:")
display(X_p4)
assert_true("Intercept" in X_p4 and len(X_p4.columns) == 8)
assert_true(X_p4.loc[[0], :].round(1).isin([1.8]).any().any())
assert_true(X_p4.loc[[3], :].round(1).isin([2.2]).any().any())

You have set X_p4 as:


,Intercept,C(c4)[T.2],C(c4)[T.3],C(c4)[T.4],r1,C(c4)[T.2]:r1,C(c4)[T.3]:r1,C(c4)[T.4]:r1
0,1.0,1.0,0.0,0.0,1.764052,1.764052,0.000000,0.000000
1,1.0,0.0,0.0,0.0,0.400157,0.000000,0.000000,0.000000
2,1.0,1.0,0.0,0.0,0.978738,0.978738,0.000000,0.000000
3,1.0,1.0,0.0,0.0,2.240893,2.240893,0.000000,0.000000
4,1.0,1.0,0.0,0.0,1.867558,1.867558,0.000000,0.000000
5,1.0,0.0,1.0,0.0,-0.977278,-0.000000,-0.977278,-0.000000
6,1.0,0.0,0.0,0.0,0.950088,0.000000,0.000000,0.000000
7,1.0,0.0,0.0,1.0,-0.151357,-0.000000,-0.000000,-0.151357


## Applying Transforms to New Data

Creating design matrices using `patsy` is very useful due to the way in which it automatically performs some standard transformations on the data (such as dummy coding categorical data or providing easy ways to standardize variables). However, `patsy` becomes truly powerful when you are trying to perform the same transformations on a new set of data, e.q. for prediction.

This is not a trivial problem due to many transformations being *stateful*. For example, when data is standardized, a particular mean and standard deviation is calculated. When you build a predictive model, the model implicitly assumes that any new data is standardized using *the same* mean and standard deviation. However, if your prediction data that your predicting on has a different mean and standard deviation and you try to standardize the data using the prediction data mean and standard deviation, your model may perform very badly.

Another way to see this is imagine that you are trying to predict on a single new row of data. With a single row of data, there is no way to standardize the data (you can't compute a standard deviation with a single row!), so you have no choice but to standardize using the mean and standard deviation from the training data.

`patsy` makes consistently applying identical transformations very easy. This is because every design matrix (i.e. `y` or `X`) that you compute comes with the *design info* for how to build that design matrix. This is stored in the attribute `.design_info` for the matrix. Let's explore this, but first, we are going to need some data with a "training set" and a "test set."

In [252]:
big_data = pd.DataFrame(demo_data("a", "b", "x1", "x2", "y", "z_column", min_rows=30))
np.random.seed(204)
big_data["c4"] = np.random.choice([1,2,3,4], len(big_data))
training_data, testing_data = train_test_split(big_data, test_size=.5, random_state=201)
print("training_data is:")
display(training_data)
print("testing_data is:")
display(testing_data)

training_data is:


,a,b,x1,x2,y,z_column,c4
17,a1,b2,-0.205158,-0.212740,0.900826,1.910065,4
2,a2,b1,0.978738,-0.347912,-1.630198,0.126912,2
13,a1,b2,0.121675,-0.438074,-0.578850,1.480515,4
8,a1,b1,-0.103219,-1.048553,1.139401,-1.173123,3
25,a1,b2,-1.454366,0.302472,1.054452,-1.099401,4
15,a2,b2,0.333674,0.777490,0.056165,0.906045,4
16,a1,b1,1.494079,-1.613898,-1.165150,-0.861226,3
3,a2,b2,2.240893,0.156349,0.462782,0.401989,2
30,a2,b1,0.154947,-0.813146,0.356366,-0.435154,1
23,a2,b2,-0.742165,0.428332,-0.179925,0.922207,3


testing_data is:


,a,b,x1,x2,y,z_column,c4
26,a2,b1,0.045759,-0.634322,-0.403177,0.298238,1
6,a2,b1,0.950088,-0.387327,0.729091,-1.270485,1
12,a1,b1,0.761038,-0.509652,-0.870797,1.922942,3
27,a2,b2,-0.187184,-0.362741,1.222445,1.326386,4
19,a2,b2,-0.854096,0.386902,-1.536244,0.802456,3
11,a2,b2,1.454274,1.950775,-0.684810,-0.747455,1
22,a2,b1,0.864436,-0.028182,1.178780,0.614079,1
7,a2,b2,-0.151357,-0.302303,0.128983,0.969397,4
10,a2,b1,0.144044,-1.706270,0.402342,-0.413619,2
14,a2,b1,0.443863,-1.252795,-0.311553,1.867559,1


Now, we can create a design matrix `X` using the training data. Then we can access the `.design_info` method to see that it has stored the design info.

In [253]:
y_training, X_training = dmatrices("np.log(y + 10) ~ center(2*x1) + standardize(2*x2)", training_data, return_type="dataframe")
display(X_training)
display(X_training.design_info)

,Intercept,center(2 * x1),standardize(2 * x2)
17,1.0,-1.446972,0.366396
2,1.0,0.920820,0.218860
13,1.0,-0.793306,0.120451
8,1.0,-1.243094,-0.545865
25,1.0,-3.945387,0.928732
15,1.0,-0.369307,1.447198
16,1.0,1.951502,-1.162920
3,1.0,3.445131,0.769244
30,1.0,-0.726761,-0.288927
23,1.0,-2.520986,1.066104


DesignInfo(['Intercept', 'center(2 * x1)', 'standardize(2 * x2)'],
           factor_infos={EvalFactor('center(2 * x1)'): FactorInfo(factor=EvalFactor('center(2 * x1)'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('standardize(2 * x2)'): FactorInfo(factor=EvalFactor('standardize(2 * x2)'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('center(2 * x1)')]),
                                      [SubtermInfo(factors=

While the `.design_info` attribute does not mean much to me, it allows us to use the function `build_design_matrices()` to apply the same transformations to the `testing_data`.

In [254]:
X_testing = build_design_matrices([X_training.design_info], testing_data, return_type="dataframe")[0]
display(X_testing)

,Intercept,center(2 * x1),standardize(2 * x2)
26,1.0,-0.945139,-0.093747
6,1.0,0.863521,0.175840
12,1.0,0.485420,0.042326
27,1.0,-1.411023,0.202675
19,1.0,-2.744847,1.020885
11,1.0,1.871891,2.727799
22,1.0,0.692217,0.567834
7,1.0,-1.339370,0.268641
10,1.0,-0.748569,-1.263741
14,1.0,-0.148929,-0.768789


There are a few funny things about the `build_design_matrices()` function. Let's look at exactly how we used it above. The command was
``` python
X_testing = build_design_matrices([X_training.design_info], testing_data, return_type="dataframe")[0]
```
First, it takes in a *list* of design infos. That is why we had the `[` and `]` around the `X_training.design_info`. It takes in a list because you could theoretically give it several design infos and get back several design matrices created from the same set of data. We then gave the function the data we wanted it to use to build the design matrix `X_testing`. Since we were building the design matrix for the testing set, we gave it the `testing_data`. We then told the function we wanted a `pandas` dataframe back. 

Finally, we have a `[0]` at the end of the function. This is because `build_design_matrices` returns a *list* of design matrices. In our invocation of the function, the list only has a single design matrix in it, but it is still a list. So, in order to get the actual design matrix out, we have to tell the list which matrix we want. Since there is only one design matrix, we tell it we want the first element in the list (which has index `0` in python). If we gave the function more design infos to work with, it would give us back more design matrices in the list, and we could choose which design matrix we wanted out of the list.

We can see why might use this if we also wanted to create a new `y_testing` design matrix as follows:

In [255]:
y_testing, X_testing = build_design_matrices([y_training.design_info, X_training.design_info], testing_data, return_type="dataframe")
print("X_testing is:")
display(X_testing)
print("y_testing is:")
display(y_testing)

X_testing is:


,Intercept,center(2 * x1),standardize(2 * x2)
26,1.0,-0.945139,-0.093747
6,1.0,0.863521,0.175840
12,1.0,0.485420,0.042326
27,1.0,-1.411023,0.202675
19,1.0,-2.744847,1.020885
11,1.0,1.871891,2.727799
22,1.0,0.692217,0.567834
7,1.0,-1.339370,0.268641
10,1.0,-0.748569,-1.263741
14,1.0,-0.148929,-0.768789


y_testing is:


,np.log(y + 10)
26,2.261432
6,2.372959
12,2.211478
27,2.417916
19,2.135793
11,2.231646
22,2.414017
7,2.315401
10,2.342031
14,2.270934


While this is very helpful when using a stateful transform like standardizing, it is also very helpful when dummy coding variables. Models expect variables to be dummy coded in exactly the same way for both the prediction and the testing set, and if you do something like switch the order of the columns for dummy coding or you drop a different column to avoid over-specification, then your model will not work.

Luckily, the same method above works for dummy coding as we can see below:

In [256]:
X_training = dmatrix("a*x1", training_data, return_type="dataframe")
display(X_training)

,Intercept,a[T.a2],x1,a[T.a2]:x1
17,1.0,0.0,-0.205158,-0.000000
2,1.0,1.0,0.978738,0.978738
13,1.0,0.0,0.121675,0.000000
8,1.0,0.0,-0.103219,-0.000000
25,1.0,0.0,-1.454366,-0.000000
15,1.0,1.0,0.333674,0.333674
16,1.0,0.0,1.494079,0.000000
3,1.0,1.0,2.240893,2.240893
30,1.0,1.0,0.154947,0.154947
23,1.0,1.0,-0.742165,-0.742165


In [257]:
X_testing = build_design_matrices([X_training.design_info], testing_data, return_type="dataframe")[0]
display(X_testing)

,Intercept,a[T.a2],x1,a[T.a2]:x1
26,1.0,1.0,0.045759,0.045759
6,1.0,1.0,0.950088,0.950088
12,1.0,0.0,0.761038,0.000000
27,1.0,1.0,-0.187184,-0.187184
19,1.0,1.0,-0.854096,-0.854096
11,1.0,1.0,1.454274,1.454274
22,1.0,1.0,0.864436,0.864436
7,1.0,1.0,-0.151357,-0.151357
10,1.0,1.0,0.144044,0.144044
14,1.0,1.0,0.443863,0.443863


Note that one thing you do have to be careful of is that your testing data has the same categories as the training data (or at least that it doesn't have any new categories, it can have fewer). If you think about this for a second, it should be obvious why. If a new category is introduced in the testing data, `patsy` (and no other data transformation procedure) can possibly know what to do with the new category without some input from the modeler. A common way to handle this is to have an `OTHER` category for your categorical variables that you use to lump together all of the new variables. This particular transformation must be done separately, and it is outside of the scope of this assignment. However, keep in mind that novel categories are going to be a problem for all data transformation techniques.

The ability to quickly, easily, and consistently apply transformations to data makes `patsy` an incredibly useful tool. There are lots of other ways to transform data using `pandas` and `sklearn`, but the combination of features and intuitive formula notation makes `patsy` a very useful tool in the modelers toolbox.

### Problem #5 - 5 point

Use the following data sets, `training_data_p5` and `testing_data_p5`, to create the design matrix `X_p5_testing` that is transformed in the same way as `X_p5_training`.

In [258]:
# This is a read only cell.
big_data_p5 = pd.DataFrame(demo_data("a", "b", "x1", "x2", "y", "z_column", min_rows=30))
np.random.seed(204)
big_data_p5["c4"] = np.random.choice([1,2,3,4], len(big_data_p5))
training_data_p5, testing_data_p5 = train_test_split(big_data_p5, test_size=.5, random_state=201)
print("training_data_p5 is:")
display(training_data_p5)
print("testing_data_p5 is:")
display(testing_data_p5)

training_data_p5 is:


,a,b,x1,x2,y,z_column,c4
17,a1,b2,-0.205158,-0.212740,0.900826,1.910065,4
2,a2,b1,0.978738,-0.347912,-1.630198,0.126912,2
13,a1,b2,0.121675,-0.438074,-0.578850,1.480515,4
8,a1,b1,-0.103219,-1.048553,1.139401,-1.173123,3
25,a1,b2,-1.454366,0.302472,1.054452,-1.099401,4
15,a2,b2,0.333674,0.777490,0.056165,0.906045,4
16,a1,b1,1.494079,-1.613898,-1.165150,-0.861226,3
3,a2,b2,2.240893,0.156349,0.462782,0.401989,2
30,a2,b1,0.154947,-0.813146,0.356366,-0.435154,1
23,a2,b2,-0.742165,0.428332,-0.179925,0.922207,3


testing_data_p5 is:


,a,b,x1,x2,y,z_column,c4
26,a2,b1,0.045759,-0.634322,-0.403177,0.298238,1
6,a2,b1,0.950088,-0.387327,0.729091,-1.270485,1
12,a1,b1,0.761038,-0.509652,-0.870797,1.922942,3
27,a2,b2,-0.187184,-0.362741,1.222445,1.326386,4
19,a2,b2,-0.854096,0.386902,-1.536244,0.802456,3
11,a2,b2,1.454274,1.950775,-0.684810,-0.747455,1
22,a2,b1,0.864436,-0.028182,1.178780,0.614079,1
7,a2,b2,-0.151357,-0.302303,0.128983,0.969397,4
10,a2,b1,0.144044,-1.706270,0.402342,-0.413619,2
14,a2,b1,0.443863,-1.252795,-0.311553,1.867559,1


In the next cell, we create a design matrix `X_p5_training` where we transform the data in various ways.

In [259]:
# This is a read only cell.
X_p5_training = dmatrix("a*b + standardize(x1) + center(I(x1 + x2)) + C(c4, Treatment(3))*z_column", training_data_p5, return_type="dataframe")
display(X_p5_training)

,Intercept,a[T.a2],b[T.b2],"C(c4, Treatment(3))[T.1]","C(c4, Treatment(3))[T.2]","C(c4, Treatment(3))[T.4]",a[T.a2]:b[T.b2],standardize(x1),center(I(x1 + x2)),z_column,"C(c4, Treatment(3))[T.1]:z_column","C(c4, Treatment(3))[T.2]:z_column","C(c4, Treatment(3))[T.4]:z_column"
17,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-0.765801,-0.387795,1.910065,0.000000,0.000000,1.910065
2,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.487338,0.660929,0.126912,0.000000,0.126912,0.000000
13,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-0.419852,-0.286296,1.480515,0.000000,0.000000,1.480515
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.657900,-1.121668,-1.173123,-0.000000,-0.000000,-0.000000
25,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-2.088072,-1.121790,-1.099401,-0.000000,-0.000000,-1.099401
15,1.0,1.0,1.0,0.0,0.0,1.0,1.0,-0.195454,1.141268,0.906045,0.000000,0.000000,0.906045
16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.032821,-0.089715,-0.861226,-0.000000,-0.000000,-0.000000
3,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.823314,2.427346,0.401989,0.000000,0.401989,0.000000
30,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.384634,-0.628095,-0.435154,-0.435154,-0.000000,-0.000000
23,1.0,1.0,1.0,0.0,0.0,0.0,1.0,-1.334216,-0.283729,0.922207,0.000000,0.000000,0.000000


Use `X_p5_training` to create the design matrix `X_p5_testing` that has the identical set of transformations applied to it. I.e., `X_p5_testing` should have the same standardization, the same centering, and the same dummy coding applied to it in the exact same way.

In [260]:
# Create the design matrix X_p5_testing
X_p5_testing = build_design_matrices([X_p5_training.design_info], testing_data_p5, return_type="dataframe")[0]
display(X_p5_testing)

,Intercept,a[T.a2],b[T.b2],"C(c4, Treatment(3))[T.1]","C(c4, Treatment(3))[T.2]","C(c4, Treatment(3))[T.4]",a[T.a2]:b[T.b2],standardize(x1),center(I(x1 + x2)),z_column,"C(c4, Treatment(3))[T.1]:z_column","C(c4, Treatment(3))[T.2]:z_column","C(c4, Treatment(3))[T.4]:z_column"
26,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.500209,-0.558460,0.298238,0.298238,0.000000,0.000000
6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.457013,0.592865,-1.270485,-1.270485,-0.000000,-0.000000
12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.256905,0.281489,1.922942,0.000000,0.000000,0.000000
27,1.0,1.0,1.0,0.0,0.0,1.0,1.0,-0.746776,-0.519821,1.326386,0.000000,0.000000,1.326386
19,1.0,1.0,1.0,0.0,0.0,0.0,1.0,-1.452694,-0.437090,0.802456,0.000000,0.000000,0.000000
11,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.990687,3.435153,-0.747455,-0.747455,-0.000000,-0.000000
22,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.366351,0.866358,0.614079,0.614079,0.000000,0.000000
7,1.0,1.0,1.0,0.0,0.0,1.0,1.0,-0.708853,-0.423556,0.969397,0.000000,0.000000,0.969397
10,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.396175,-1.532123,-0.413619,-0.000000,-0.413619,-0.000000
14,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.078820,-0.778828,1.867559,1.867559,0.000000,0.000000


In [261]:
# THIS IS A GRADING CELL. DO NOT EDIT AND DO NOT COPY.
from nose.tools import assert_equal, assert_true
import numpy as np
print("You have set X_p5_testing as:")
display(X_p5_testing)
assert_true((X_p5_testing.columns == X_p5_training.columns).all())
assert_true(X_p5_testing.loc[[26], :].round(1).isin([-0.6]).any().any())
assert_true(X_p5_testing.loc[[10], :].sum().sum().round(1) == 0.2)

You have set X_p5_testing as:


,Intercept,a[T.a2],b[T.b2],"C(c4, Treatment(3))[T.1]","C(c4, Treatment(3))[T.2]","C(c4, Treatment(3))[T.4]",a[T.a2]:b[T.b2],standardize(x1),center(I(x1 + x2)),z_column,"C(c4, Treatment(3))[T.1]:z_column","C(c4, Treatment(3))[T.2]:z_column","C(c4, Treatment(3))[T.4]:z_column"
26,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.500209,-0.558460,0.298238,0.298238,0.000000,0.000000
6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.457013,0.592865,-1.270485,-1.270485,-0.000000,-0.000000
12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.256905,0.281489,1.922942,0.000000,0.000000,0.000000
27,1.0,1.0,1.0,0.0,0.0,1.0,1.0,-0.746776,-0.519821,1.326386,0.000000,0.000000,1.326386
19,1.0,1.0,1.0,0.0,0.0,0.0,1.0,-1.452694,-0.437090,0.802456,0.000000,0.000000,0.000000
11,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.990687,3.435153,-0.747455,-0.747455,-0.000000,-0.000000
22,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.366351,0.866358,0.614079,0.614079,0.000000,0.000000
7,1.0,1.0,1.0,0.0,0.0,1.0,1.0,-0.708853,-0.423556,0.969397,0.000000,0.000000,0.969397
10,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.396175,-1.532123,-0.413619,-0.000000,-0.413619,-0.000000
14,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.078820,-0.778828,1.867559,1.867559,0.000000,0.000000
